In [1]:
import pandas as pd
# ================== Preprocessing ====================
# Set the name of the file
rating_filename = 'Ratingst.csv'
# user_filename = 'Users.csv'
# book_filename = 'Books.csv'

ratings_df = pd.read_csv(rating_filename)

print(ratings_df.head())

# Rating - User-ID, ISBN, Book-Rating
# Connect via ISBN

   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726   155061224            5
2   276727   446520802            0
3   276729  052165615X            3
4   276729   521795028            6


In [2]:
print(len(ratings_df["User-ID"].unique()))
print(len(ratings_df["ISBN"].unique()))
print(ratings_df.shape)

12
26
(26, 3)


In [3]:
# Take the aggregate ratings (worst case scenario if user is new and age is unknown)
avg_ratings = ratings_df.groupby('ISBN').agg(avg_rating = ('Book-Rating', 'mean'),
                                                number_of_ratings = ('Book-Rating', 'count')).reset_index()

# Keep the books with over 100 ratings
avg_ratings100 = avg_ratings[avg_ratings['number_of_ratings']>100]

In [4]:
# Check popular books
avg_ratings100.sort_values(by='avg_rating', ascending=False).head()

,ISBN,avg_rating,number_of_ratings


In [5]:
# Drop book ratings with 0 because it is already implied and not every book has a rating so it is unnecessary
ratings_df = ratings_df[ratings_df["Book-Rating"] != 0]
ratings_df.shape

(13, 3)

In [6]:
import numpy as np
#ratings = np.array(ratings_df["Book-Rating"])
unique_users = np.array(ratings_df["User-ID"].sort_values().unique())
# unique_books = np.array(ratings_df["ISBN"].sort_values().unique())

In [7]:
# Number of ratings not equal for all, should remove 0's
user_counts = ratings_df.groupby('User-ID').agg(number_of_ratings = ('ISBN', 'count')).reset_index()
user_counts.head()

,User-ID,number_of_ratings
0,276726,1
1,276729,2
2,276736,1
3,276737,1
4,276744,1


In [8]:
print(user_counts.mean())
print(user_counts.max())

User-ID              276739.000
number_of_ratings         1.625
dtype: float64
User-ID              276748
number_of_ratings         5
dtype: int64


In [9]:
user_book = dict()

#book_index = {k: v for v, k in enumerate(unique_books)}
user_index = {k: v for v, k in enumerate(unique_users)}

In [10]:
arr_users = [{} for _ in range(len(unique_users))]
i = 0
for user_id, group in ratings_df.groupby("User-ID"):
    for index, row in group.iterrows():
        arr_users[i][row['ISBN']] = row['Book-Rating']
    i += 1

In [11]:
arr_users

[{'155061224': 5},
 {'052165615X': 3, '521795028': 6},
 {'3257224281': 8},
 {'600570967': 6},
 {'038550120X': 7},
 {'342310538': 10},
 {'60517794': 9,
  '671537458': 9,
  '679776818': 8,
  '943066433': 7,
  '1885408226': 7},
 {'747558167': 6}]

In [12]:
neighbors = dict()

In [13]:
vals = list(set(arr_users[1].keys()).symmetric_difference(set(arr_users[0].keys())))
vals

['155061224', '521795028', '052165615X']

In [16]:
for keys in arr_users[1].keys():
    print(keys)

052165615X
521795028


In [27]:
# for i in range(len(arr_users)):
#     for j in range(len(arr_users[i+1:])):
#         vals = list(set(arr_users[i].keys()).symmetric_difference(set(arr_users[j].keys())))
#         arr1 = [0] * len(vals)
#         arr2 = [0] * len(vals)
#         for k in range(len(vals)):
#             arr1[k] = arr_users[i].get(vals.)

In [12]:
# Iterate through the grouped DataFrame
for user_id, group in ratings_df.groupby("User-ID"):
    user_ratings = [0] * len(unique_books)
    for index, row in group.iterrows():
        # Access individual rows using 'row'
        user_ratings[book_index.get(row['ISBN'])] = row['Book-Rating']
        #print(f"User-ID: {user_id}, ISBN: {row['ISBN']}, Rating: {row['Book-Rating']}")
    user_book[user_id] = user_ratings

MemoryError: 

In [14]:
user_book.get(276729)

[0, 3, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0]

In [69]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

rcLabel, vLabel = ('User-ID', 'ISBN'), 'Book-Rating'
rcCat = [CategoricalDtype(sorted(df[col].unique()), ordered=True) for col in rcLabel]
rc = [df[column].astype(aType).cat.codes for column, aType in zip(rcLabel, rcCat)]
mat = csr_matrix((df[vLabel], rc), shape=tuple(cat.categories.size for cat in rcCat))
dfOut = ( pd.DataFrame.sparse.from_spmatrix(
    mat, index=rcCat[0].categories, columns=rcCat[1].categories) )